In [29]:
import io
import os
import pickle
from pathlib import Path
import pandas as pd
import torch
import shutil
from utils import str_to_dict, forward_model_from_str, psnr_from_mse, psnr

In [30]:
root = './runs/results/mgan_biggan_inv'
image_size = '128'
model = 'mgan_biggan'
rows_list = []
p = Path(root)

In [ ]:
for cuts_path in p.iterdir():
    print(cuts_path.name)
    cuts_comb = cuts_path.name.split('=')[1]
    
    
    for split_path in cuts_path.iterdir():
        split = split_path.name
        # split = ood-examples
        
        for image_name_path in split_path.iterdir():
            image_name = image_name_path.name
#             orig_img = load_target_image(os.path.join(args.img_dir, img_name),
#                                      img_size).to(DEVICE)
            
            for fm_path in image_name_path.iterdir():
                fm_name, fm_params = forward_model_from_str(
                    fm_path.name)
                for params_path in fm_path.iterdir():
                    if not os.path.exists(
                            params_path / 'metadata.pkl'):
#                         print('metadata.pkl not found for',
#                               params_path)
                        continue
                        
                    with open(params_path / 'metadata.pkl', 'rb') as f:
                        metadata = pickle.load(f)
                        
                    if os.path.exists(params_path /
                                      'psnr_clamped.pkl'):
                        with open(params_path / 'psnr_clamped.pkl',
                                  'rb') as f:
                            ps = pickle.load(f)
                    else:
#                         print("Calculating PSNR for original image")
                        recovered = torch.load(str(params_path /
                                                   'recovered.pt'),
                                               map_location='cpu')
                        orig_path = p.parent.parent.joinpath(
                            'images', split, image_name, image_size,
                            'original.pt')
                        orig = torch.load(str(orig_path),
                                          map_location='cpu')

                        if recovered.dim() == 4:
                            recovered = recovered.squeeze()
                            torch.save(recovered,
                                       params_path / 'recovered.pt')
                        
                        if recovered.shape[1] == 512:
                            print("Corrupted! Removing...")
                            shutil.rmtree(params_path)
                            continue
                        
                            
                        ps = psnr(orig, recovered.clamp(0, 1))

                        with open(params_path / 'psnr_clamped.pkl',
                                  'wb') as f:
                            pickle.dump(float(ps), f)

                    current_row_dict = {}

                    current_row_dict['model'] = model
                    current_row_dict['cuts_comb'] = cuts_comb
                    current_row_dict['split'] = split
                    current_row_dict['image_name'] = image_name
                    current_row_dict['fm'] = fm_name
                    current_row_dict['fraction_kept'] = float(
                        fm_params.get('fraction_kept', -1.))
                    current_row_dict['scale_factor'] = float(
                        fm_params.get('scale_factor', -1.))
                    current_row_dict.update(metadata)
                    current_row_dict['psnr'] = float(ps)

                    rows_list.append(current_row_dict)

df = pd.DataFrame(rows_list)
os.makedirs(p.parent / 'processed_results', exist_ok=True)
with open(p.parent / 'processed_results/df_results.pkl', 'wb') as f:
    pickle.dump(df, f)

In [34]:
with open('./runs/results/processed_results/df_results.pkl', 'rb') as f:
    df = pickle.load(f)

In [112]:
pd.set_option('display.max_rows', None) 

In [ ]:
sorted(df['cuts_comb'].unique())

In [82]:
inpaint = df[(df['fm'] == 'InpaintingScatter') & (df['restarts'] == 1) & (df['n_steps'] == 30)]

In [90]:
inpaint = inpaint.drop_duplicates()

In [94]:
inpaint.sort_values('psnr',ascending = False)

,model,cuts_comb,split,image_name,fm,fraction_kept,scale_factor,optimizer,n_steps,z_lr,z_number,restarts,first_cut,second_cut,z_init_mode,limit,psnr
13,mgan_biggan,"[4, 11]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,4,11,clamped_normal,1,19.631826
30,mgan_biggan,"[1, 11]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,1,11,clamped_normal,1,18.749525
28,mgan_biggan,"[5, 8]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,5,8,clamped_normal,1,18.712786
146,mgan_biggan,"[2, 3]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,2,3,clamped_normal,1,18.502062
52,mgan_biggan,"[3, 4]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,3,4,clamped_normal,1,18.408350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,mgan_biggan,"[0, 7]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,0,7,clamped_normal,1,4.472544
153,mgan_biggan,"[7, 8]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,7,8,clamped_normal,1,4.329803
122,mgan_biggan,"[2, 6]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,2,6,clamped_normal,1,3.743235
85,mgan_biggan,"[3, 8]",ood-examples,surfer,InpaintingScatter,0.05,-1.0,lbfgs,30,1.0,20,1,3,8,clamped_normal,1,3.742792


In [95]:
res = df[(df['fm'] == 'SuperResolution') & (df['restarts'] == 1) & (df['n_steps'] == 30)]

In [97]:
res = res.drop_duplicates()

In [98]:
res.sort_values('psnr',ascending = False)

,model,cuts_comb,split,image_name,fm,fraction_kept,scale_factor,optimizer,n_steps,z_lr,z_number,restarts,first_cut,second_cut,z_init_mode,limit,psnr
151,mgan_biggan,"[2, 11]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,2,11,clamped_normal,1,20.153019
31,mgan_biggan,"[1, 11]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,1,11,clamped_normal,1,20.097799
76,mgan_biggan,"[3, 11]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,3,11,clamped_normal,1,20.037470
500,mgan_biggan,"[6, 8]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,6,8,clamped_normal,1,18.820322
37,mgan_biggan,"[1, 3]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,1,3,clamped_normal,1,18.814987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,mgan_biggan,"[0, 3]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,0,3,clamped_normal,1,13.185988
43,mgan_biggan,"[0, 7]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,0,7,clamped_normal,1,11.567926
83,mgan_biggan,"[0, 8]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,0,8,clamped_normal,1,10.694648
506,mgan_biggan,"[0, 5]",ood-examples,surfer,SuperResolution,-1.0,0.25,lbfgs,30,1.0,20,1,0,5,clamped_normal,1,4.730254


In [99]:
sq = df[(df['fm'] == 'InpaintingSquare') & (df['restarts'] == 1) & (df['n_steps'] == 30)]

In [100]:
sq = sq.drop_duplicates()

In [101]:
sq.sort_values('psnr',ascending = False)

,model,cuts_comb,split,image_name,fm,fraction_kept,scale_factor,optimizer,n_steps,z_lr,z_number,restarts,first_cut,second_cut,z_init_mode,limit,psnr
12,mgan_biggan,"[4, 11]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,4,11,clamped_normal,1,27.946703
74,mgan_biggan,"[3, 11]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,3,11,clamped_normal,1,27.822523
71,mgan_biggan,"[1, 10]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,1,10,clamped_normal,1,27.645014
47,mgan_biggan,"[2, 10]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,2,10,clamped_normal,1,27.614145
18,mgan_biggan,"[1, 13]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,1,13,clamped_normal,1,27.249491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,mgan_biggan,"[1, 6]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,1,6,clamped_normal,1,3.743236
57,mgan_biggan,"[11, 12]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,11,12,clamped_normal,1,3.743235
141,mgan_biggan,"[0, 9]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,0,9,clamped_normal,1,3.743235
134,mgan_biggan,"[9, 10]",ood-examples,surfer,InpaintingSquare,-1.0,-1.0,lbfgs,30,1.0,20,1,9,10,clamped_normal,1,3.743235


In [124]:
df[(df['restarts'] == 1) & (df['n_steps'] == 30)].drop_duplicates().groupby('cuts_comb').mean().sort_values('psnr', ascending=True).head(20)

,fraction_kept,scale_factor,n_steps,z_lr,z_number,restarts,first_cut,second_cut,limit,psnr
cuts_comb,,,,,,,,,,
"[9, 10]",-1.000,-1.000000,30.0,1.0,20.0,1.0,9.0,10.0,1.0,3.743235
"[11, 12]",-1.000,-1.000000,30.0,1.0,20.0,1.0,11.0,12.0,1.0,3.743235
"[9, 12]",-1.000,-1.000000,30.0,1.0,20.0,1.0,9.0,12.0,1.0,3.743981
"[0, 4]",-1.000,-1.000000,30.0,1.0,20.0,1.0,0.0,4.0,1.0,4.297445
"[7, 8]",-0.475,-1.000000,30.0,1.0,20.0,1.0,7.0,8.0,1.0,4.357591
"[0, 5]",-0.650,-0.583333,30.0,1.0,20.0,1.0,0.0,5.0,1.0,4.647686
"[6, 9]",-0.650,-0.583333,30.0,1.0,20.0,1.0,6.0,9.0,1.0,9.052649
"[1, 6]",-0.650,-0.583333,30.0,1.0,20.0,1.0,1.0,6.0,1.0,11.971258
"[5, 6]",-0.650,-0.583333,30.0,1.0,20.0,1.0,5.0,6.0,1.0,12.385658
